<a href="https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/bootcamp/tutorials/integration/qa_with_milvus_and_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>   <a href="https://github.com/milvus-io/bootcamp/blob/master/bootcamp/tutorials/integration/qa_with_milvus_and_hf.ipynb" target="_blank">
    <img src="https://img.shields.io/badge/View%20on%20GitHub-555555?style=flat&logo=github&logoColor=white" alt="GitHub Repository"/>
</a>

# Question Answering Using Milvus and Hugging Face

A question answering system based on semantic search works by finding the most similar question from a dataset of question-answer pairs for a given query question. Once the most similar question is identified, the corresponding answer from the dataset is considered as the answer for the query. This approach relies on semantic similarity measures to determine the similarity between questions and retrieve relevant answers.

This tutorial shows how to build a question answering system using [Hugging Face](https://huggingface.co) as the data loader & embedding generator for data processing and [Milvus](https://milvus.io) as the vector database for semantic search.

## Before you begin

You need to make sure all required dependencies are installed:

- `pymilvus`: a python package works with the vector database service powered by Milvus or Zilliz Cloud.
- `datasets`, `transformers`: Hugging Face packages manage data and utilize models.
- `torch`: a powerful library provides efficient tensor computation and deep learning tools.

In [1]:
%pip install --upgrade pymilvus transformers datasets torch==2.5.1

> If you are using Google Colab, to enable dependencies just installed, you may need to **restart the runtime** (click on the "Runtime" menu at the top of the screen, and select "Restart session" from the dropdown menu).

## Prepare data

In this section, we will load example question-answer pairs from the Hugging Face Datasets. As a demo, we only take partial data from the validation split of [SQuAD](https://huggingface.co/datasets/rajpurkar/squad).

In [21]:
from datasets import load_dataset


DATASET = "squad"  # Name of dataset from HuggingFace Datasets
INSERT_RATIO = 100.0  # Ratio of example dataset to be inserted

data = load_dataset(DATASET, split="validation")
# Generates a fixed subset. To generate a random subset, remove the seed.
data = data #.train_test_split(test_size=INSERT_RATIO)["test"]
# Clean up the data structure in the dataset.
data = data.map(
    lambda val: {"answer": val["answers"]["text"][0]},
    remove_columns=["id", "answers", "context"],
)

# View summary of example data
print(data)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'question', 'answer'],
    num_rows: 10570
})


To generate embeddings for questions, you are able to select a text embedding model from Hugging Face Models. In this tutorial, we will use a small sentencce embedding model [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) as example.

In [22]:
from transformers import AutoTokenizer, AutoModel
import torch

MODEL = (
    "sentence-transformers/all-MiniLM-L6-v2"  # Name of model from HuggingFace Models
)
INFERENCE_BATCH_SIZE = 64  # Batch size of model inference

# Load tokenizer & model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModel.from_pretrained(MODEL)


def encode_text(batch):
    # Tokenize sentences
    encoded_input = tokenizer(
        batch["question"], padding=True, truncation=True, return_tensors="pt"
    )

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    token_embeddings = model_output[0]
    attention_mask = encoded_input["attention_mask"]
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    sentence_embeddings = torch.sum(
        token_embeddings * input_mask_expanded, 1
    ) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Normalize embeddings
    batch["question_embedding"] = torch.nn.functional.normalize(
        sentence_embeddings, p=2, dim=1
    )
    return batch


data = data.map(encode_text, batched=True, batch_size=INFERENCE_BATCH_SIZE)
data_list = data.to_list()

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

## Insert data

Now we have question-answer pairs ready with question embeddings. The next step is to insert them into the vector database.

We will first need to connect to Milvus service and create a Milvus collection.

In [23]:
from pymilvus import MilvusClient


MILVUS_URI = "./huggingface_milvus_test.db"  # Connection URI
COLLECTION_NAME = "huggingface_test"  # Collection name
DIMENSION = 384  # Embedding dimension depending on model

milvus_client = MilvusClient(MILVUS_URI)
if milvus_client.has_collection(collection_name=COLLECTION_NAME):
    milvus_client.drop_collection(collection_name=COLLECTION_NAME)
milvus_client.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=DIMENSION,
    auto_id=True,  # Enable auto id
    enable_dynamic_field=True,  # Enable dynamic fields
    vector_field_name="question_embedding",  # Map vector field name and embedding column in dataset
    consistency_level="Strong",  # To enable search with latest data
)

> As for the argument of `MilvusClient`:
> - Setting the `uri` as a local file, e.g.`./milvus.db`, is the most convenient method, as it automatically utilizes [Milvus Lite](https://milvus.io/docs/milvus_lite.md) to store all data in this file.
> - If you have large scale of data, you can set up a more performant Milvus server on [docker or kubernetes](https://milvus.io/docs/quickstart.md). In this setup, please use the server uri, e.g.`http://localhost:19530`, as your `uri`.
> - If you want to use [Zilliz Cloud](https://zilliz.com/cloud), the fully managed cloud service for Milvus, adjust the `uri` and `token`, which correspond to the [Public Endpoint and Api key](https://docs.zilliz.com/docs/on-zilliz-cloud-console#free-cluster-details) in Zilliz Cloud.

Insert all data into the collection:

In [ ]:
milvus_client.insert(collection_name=COLLECTION_NAME, data=data_list)

## Ask questions

Once all the data is inserted into Milvus, we can ask questions and see what the closest answers are.

In [34]:
questions = {
    "question": [
        "eurpoens conquest of africa"
    ]
}

# Generate question embeddings
question_embeddings = [v.tolist() for v in encode_text(questions)["question_embedding"]]

# Search across Milvus
search_results = milvus_client.search(
    collection_name=COLLECTION_NAME,
    data=question_embeddings,
    limit=10,  # How many search results to output
    output_fields=["answer", "question"],  # Include these fields in search results
)

# Print out results
for q, res in zip(questions["question"], search_results):
    print("Question:", q)
    for r in res:
        print(
            {
                "answer": r["entity"]["answer"],
                "score": r["distance"],
                "original question": r["entity"]["question"],
            }
        )
    print("\n")

Question: eurpoens conquest of africa
{'answer': 'blank spaces on contemporary maps', 'score': 0.6491098999977112, 'original question': 'What provided an incentive to western empires to colonize Africa?'}
{'answer': 'French', 'score': 0.6239544749259949, 'original question': 'Who besides the british colonized Africa?'}
{'answer': 'By the first millennium AD', 'score': 0.5458090901374817, 'original question': 'When did the Bantu expansion reach the area from West-Central Africa?'}
{'answer': 'rapid expansion in telecommunication and financial activity', 'score': 0.5434884428977966, 'original question': "What was East and Central Africa's economy boosted by?"}
{'answer': 'Dutch Cape Colony', 'score': 0.5064941644668579, 'original question': 'What area in South Africa accepted Huguenot colonists?'}
{'answer': 'Lower Paleolithic period', 'score': 0.49618497490882874, 'original question': 'How long has the African Great Lakes region been inhabited?'}
{'answer': 'Kenya', 'score': 0.489571571